In [2]:
import numpy as np
from tensorflow import set_random_seed
from numpy.random import seed
import pandas as pd
#import matplotlib.pyplot as plt
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from datetime import datetime
from datetime import timedelta

In [3]:
#dataset_empty = 0
set_random_seed(1)
seed(1)

In [4]:
#Convert the train data into time series
def time_series_data(data, window, lag):
    dropnan = True
    cols, names = list(), list()

    for i in range(window, 0, -1):
        #past time series data (t-)
        cols.append(data.shift(i))
        names = names + [('%s(t-%d)' % (col, i)) for col in data.columns]
    
    #current time series data (t = 0)
    cols.append(data)
    names = names + [('%s(t)' % (col)) for col in data.columns]
    
    #future data (t + lag)
    cols.append(data.shift(-lag))
    names = names + [('%s(t+%d)' % (col, lag)) for col in data.columns]
    
    #all data
    all_data = pd.concat(cols, axis=1)
    all_data.columns = names
    
    #drops rows with NaN
    if dropnan:
        all_data.dropna(inplace=True)
    return all_data

In [5]:
dataset = pd.read_csv('sales data-set.csv', usecols = ['Store', 'Item', 'Date', 'Weekly_Sales'])
train = dataset[['Store', 'Item', 'Date', 'Weekly_Sales']]
#train = train.loc[(train['Store'] == 1) & (train['Item'] == 1)]
#train = train[(train['Date'] >= '2012-01-01')]
#train = train.loc[train['Store'] == 1]

print(train.head())

   Store  Item        Date  Weekly_Sales
0      1     1  2010-02-07      24924.50
1      1     2  2010-02-07      50605.27
2      1     3  2010-02-07      13740.12
3      1     4  2010-02-07      39954.04
4      1     5  2010-02-07      32229.38


In [6]:
#Re-arranges the train dataset to apply shift methods
train_r = train.sort_values('Date').groupby(['Item', 'Store', 'Date'], as_index=False)
train_r = train_r.agg({'Weekly_Sales':['mean']})
train_r.columns = ['Item', 'Store', 'Date', 'Weekly_Sales']
#cols_to_drop = ['Date']
#train_r.drop(cols_to_drop, axis=1, inplace=True)
train_r.head()

,Item,Store,Date,Weekly_Sales
0,1,1,2010-02-07,24924.50
1,1,1,2010-02-14,46039.49
2,1,1,2010-02-21,41595.55
3,1,1,2010-02-28,19403.54
4,1,1,2010-03-07,21827.90


In [7]:
train_r = train_r.groupby(['Date'], as_index=False)['Weekly_Sales'].sum()
train_r.head()

,Date,Weekly_Sales
0,2010-02-07,49750740.50
1,2010-02-14,48336677.63
2,2010-02-21,48276993.78
3,2010-02-28,43968571.13
4,2010-03-07,46871470.30


In [8]:
#weekly sales in 100 million dollars
train_r['Weekly_Sales'] = round(train_r['Weekly_Sales'] / 10000000, 2)
train_r.head()

,Date,Weekly_Sales
0,2010-02-07,4.98
1,2010-02-14,4.83
2,2010-02-21,4.83
3,2010-02-28,4.40
4,2010-03-07,4.69


In [9]:
#the model will use last 117 weekly sales data and 
#current timestep (7 days) to forecast next weekly sales data 12 weeks ahead

#the model will use last 29 weekly sales data and 
#current timestep (7 days) to forecast next weekly sales data 4 weeks ahead

window = 117
lag = 12

series_data = time_series_data(train_r, window, lag)
series_data.head()

,Date(t-117),Weekly_Sales(t-117),Date(t-116),Weekly_Sales(t-116),Date(t-115),Weekly_Sales(t-115),Date(t-114),Weekly_Sales(t-114),Date(t-113),Weekly_Sales(t-113),...,Date(t-3),Weekly_Sales(t-3),Date(t-2),Weekly_Sales(t-2),Date(t-1),Weekly_Sales(t-1),Date(t),Weekly_Sales(t),Date(t+12),Weekly_Sales(t+12)
117,2010-02-07,4.98,2010-02-14,4.83,2010-02-21,4.83,2010-02-28,4.40,2010-03-07,4.69,...,2012-04-15,4.66,2012-04-22,4.51,2012-04-29,4.37,2012-05-06,4.71,2012-07-29,4.41
118,2010-02-14,4.83,2010-02-21,4.83,2010-02-28,4.40,2010-03-07,4.69,2010-03-14,4.59,...,2012-04-22,4.51,2012-04-29,4.37,2012-05-06,4.71,2012-05-13,4.69,2012-08-05,4.75
119,2010-02-21,4.83,2010-02-28,4.40,2010-03-07,4.69,2010-03-14,4.59,2010-03-21,4.50,...,2012-04-29,4.37,2012-05-06,4.71,2012-05-13,4.69,2012-05-20,4.68,2012-08-12,4.74
120,2010-02-28,4.40,2010-03-07,4.69,2010-03-14,4.59,2010-03-21,4.50,2010-03-28,4.41,...,2012-05-06,4.71,2012-05-13,4.69,2012-05-20,4.68,2012-05-27,4.79,2012-08-19,4.74
121,2010-03-07,4.69,2010-03-14,4.59,2010-03-21,4.50,2010-03-28,4.41,2010-04-04,5.04,...,2012-05-13,4.69,2012-05-20,4.68,2012-05-27,4.79,2012-06-03,4.83,2012-08-26,4.74


In [10]:
future_dates = series_data[['Date(t+%d)' % lag]]
future_dates

,Date(t+12)
117,2012-07-29
118,2012-08-05
119,2012-08-12
120,2012-08-19
121,2012-08-26
122,2012-09-02
123,2012-09-09
124,2012-09-16
125,2012-09-23
126,2012-09-30


In [11]:
#drops last record of (t + lag)
#last_record_item = 'Item(t-%d)' % window
#last_record_store = 'Store(t-%d)' % window
#series_data = series_data[(series_data['Item(t)'] == series_data[last_record_item])]
#series_data = series_data[(series_data['Store(t)'] == series_data[last_record_store])]

#drops Item and Store columns
cols_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Date']]
for i in range(window, 0, -1):
    cols_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Date']]

series_data.drop(cols_to_drop, axis=1, inplace=True)
series_data.drop(['Date(t)'], axis=1, inplace=True)

lbls_col = 'Weekly_Sales(t+%d)' % lag
lbls = series_data[lbls_col]
series_data = series_data.drop(lbls_col, axis=1)

series_data.head()

,Weekly_Sales(t-117),Weekly_Sales(t-116),Weekly_Sales(t-115),Weekly_Sales(t-114),Weekly_Sales(t-113),Weekly_Sales(t-112),Weekly_Sales(t-111),Weekly_Sales(t-110),Weekly_Sales(t-109),Weekly_Sales(t-108),...,Weekly_Sales(t-9),Weekly_Sales(t-8),Weekly_Sales(t-7),Weekly_Sales(t-6),Weekly_Sales(t-5),Weekly_Sales(t-4),Weekly_Sales(t-3),Weekly_Sales(t-2),Weekly_Sales(t-1),Weekly_Sales(t)
117,4.98,4.83,4.83,4.40,4.69,4.59,4.50,4.41,5.04,4.74,...,4.69,4.75,4.69,4.50,4.53,5.35,4.66,4.51,4.37,4.71
118,4.83,4.83,4.40,4.69,4.59,4.50,4.41,5.04,4.74,4.52,...,4.75,4.69,4.50,4.53,5.35,4.66,4.51,4.37,4.71,4.69
119,4.83,4.40,4.69,4.59,4.50,4.41,5.04,4.74,4.52,4.47,...,4.69,4.50,4.53,5.35,4.66,4.51,4.37,4.71,4.69,4.68
120,4.40,4.69,4.59,4.50,4.41,5.04,4.74,4.52,4.47,4.37,...,4.50,4.53,5.35,4.66,4.51,4.37,4.71,4.69,4.68,4.79
121,4.69,4.59,4.50,4.41,5.04,4.74,4.52,4.47,4.37,4.85,...,4.53,5.35,4.66,4.51,4.37,4.71,4.69,4.68,4.79,4.83


In [12]:
series_data.shape[0]

14

In [13]:
#train and test split

X_train_1, X_test_1, Y_train_1, Y_test_1 = train_test_split(series_data, lbls.values, test_size=0.1, random_state=0)
X_train = X_train_1
Y_train = Y_train_1

X_train_1, X_test_1, Y_train_1, Y_test_1 = train_test_split(series_data, lbls.values, test_size=0.5, random_state=0)
X_test = X_test_1
Y_test = Y_test_1

print('Train dataset:', X_train.shape)
print('Test dataset:', X_test.shape)
X_train.head()

Train dataset: (12, 118)
Test dataset: (7, 118)


,Weekly_Sales(t-117),Weekly_Sales(t-116),Weekly_Sales(t-115),Weekly_Sales(t-114),Weekly_Sales(t-113),Weekly_Sales(t-112),Weekly_Sales(t-111),Weekly_Sales(t-110),Weekly_Sales(t-109),Weekly_Sales(t-108),...,Weekly_Sales(t-9),Weekly_Sales(t-8),Weekly_Sales(t-7),Weekly_Sales(t-6),Weekly_Sales(t-5),Weekly_Sales(t-4),Weekly_Sales(t-3),Weekly_Sales(t-2),Weekly_Sales(t-1),Weekly_Sales(t)
121,4.69,4.59,4.50,4.41,5.04,4.74,4.52,4.47,4.37,4.85,...,4.53,5.35,4.66,4.51,4.37,4.71,4.69,4.68,4.79,4.83
128,4.47,4.37,4.85,4.53,4.51,4.78,5.02,4.78,4.76,4.66,...,4.68,4.79,4.83,4.97,4.84,4.77,4.66,5.13,4.61,4.61
119,4.83,4.40,4.69,4.59,4.50,4.41,5.04,4.74,4.52,4.47,...,4.69,4.50,4.53,5.35,4.66,4.51,4.37,4.71,4.69,4.68
130,4.85,4.53,4.51,4.78,5.02,4.78,4.76,4.66,4.89,4.79,...,4.83,4.97,4.84,4.77,4.66,5.13,4.61,4.61,4.41,4.75
126,4.74,4.52,4.47,4.37,4.85,4.53,4.51,4.78,5.02,4.78,...,4.71,4.69,4.68,4.79,4.83,4.97,4.84,4.77,4.66,5.13


In [14]:
X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_series = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))
print('Train dataset:', X_train_series.shape)
print('Test dataset:', X_test_series.shape)

Train dataset: (12, 118, 1)
Test dataset: (7, 118, 1)


In [15]:
#CNN-LSTM
subsequences = 2
time_steps = X_train_series.shape[1] // subsequences
X_train_series_sub = X_train_series.reshape((X_train_series.shape[0], subsequences, time_steps, 1))
X_test_series_sub = X_test_series.reshape((X_test_series.shape[0], subsequences, time_steps, 1))
print('Train dataset:', X_train_series_sub.shape)
print('Test dataset:', X_test_series_sub.shape)

Train dataset: (12, 2, 59, 1)
Test dataset: (7, 2, 59, 1)


In [32]:
epochs = 15000
batch = 32
learning_rate = 0.00000001
adam = optimizers.Adam(learning_rate)

In [33]:
model_cnn_lstm = Sequential()
model_cnn_lstm.add(TimeDistributed(Conv1D(filters = 64, kernel_size = 1, activation = 'relu'), input_shape = (None, X_train_series_sub.shape[2], X_train_series_sub.shape[3])))
model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model_cnn_lstm.add(TimeDistributed(Flatten()))
model_cnn_lstm.add(LSTM(60, activation = 'relu'))
model_cnn_lstm.add(Dropout(0.1))
#model_cnn_lstm.add(Dense(100, activation='relu'))
#model_cnn_lstm.add(Dropout(0.4))
#model_cnn_lstm.add(Dense(50, activation='relu'))
#model_cnn_lstm.add(Dropout(0.3))
#model_cnn_lstm.add(Dense(32, activation='relu'))
#model_cnn_lstm.add(Dense(16, activation='relu'))
model_cnn_lstm.add(Dense(30, activation = 'relu'))
model_cnn_lstm.add(Dropout(0.1))
model_cnn_lstm.add(Dense(1))
model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')
model_cnn_lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_13 (TimeDis (None, None, 59, 64)      128       
_________________________________________________________________
time_distributed_14 (TimeDis (None, None, 29, 64)      0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, None, 1856)        0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 60)                460080    
_________________________________________________________________
dropout_9 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 30)                1830      
_________________________________________________________________
dropout_10 (Dropout)         (None, 30)                0         
__________

In [ ]:
file_name = "cnn_lstm_weights_store_all.hdf5"    #saves file to create checkpoints for each epoch
checkpoint = ModelCheckpoint(file_name, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')
callbacks_list = [checkpoint]

print('Please wait...')

cnn_lstm = model_cnn_lstm.fit(X_train_series_sub, Y_train, validation_data = (X_test_series_sub, Y_test), epochs = epochs, verbose = 1, callbacks = callbacks_list, batch_size = batch)

Please wait...
Train on 12 samples, validate on 7 samples
Epoch 1/15000
12/12 [==============================] - 1s 83ms/step - loss: 20.2491 - val_loss: 8.2608

Epoch 00001: loss improved from inf to 20.24912, saving model to cnn_lstm_weights_store_all.hdf5
Epoch 2/15000
12/12 [==============================] - 0s 1ms/step - loss: 7.0981 - val_loss: 0.0699

Epoch 00002: loss improved from 20.24912 to 7.09808, saving model to cnn_lstm_weights_store_all.hdf5
Epoch 3/15000
12/12 [==============================] - 0s 1ms/step - loss: 0.7322 - val_loss: 6.7148

Epoch 00003: loss improved from 7.09808 to 0.73218, saving model to cnn_lstm_weights_store_all.hdf5
Epoch 4/15000
12/12 [==============================] - 0s 1ms/step - loss: 8.9204 - val_loss: 1.6336

Epoch 00004: loss did not improve from 0.73218
Epoch 5/15000
12/12 [==============================] - 0s 2ms/step - loss: 2.2157 - val_loss: 0.1243

Epoch 00005: loss did not improve from 0.73218
Epoch 6/15000
12/12 [=================

In [35]:
#load from saved file
file_name = "cnn_lstm_weights_store_all.hdf5"

model_cnn_lstm.load_weights(file_name)
model_cnn_lstm.compile(loss = 'mse', optimizer = 'adam')

#prediction
cnn_lstm_train_prediction = model_cnn_lstm.predict(X_train_series_sub)
cnn_lstm_test_prediction = model_cnn_lstm.predict(X_test_series_sub)

cnn_lstm_train_prediction

array([[4.695902 ],
       [4.597597 ],
       [4.690237 ],
       [4.516028 ],
       [4.3849134],
       [4.7063494],
       [4.4033394],
       [4.745607 ],
       [4.687485 ],
       [4.3956194],
       [4.684357 ],
       [4.4685616]], dtype=float32)

In [36]:
print('Train Error (RMSE):', np.sqrt(mean_squared_error(Y_train, cnn_lstm_train_prediction)))
print('Test Error (RMSE):', np.sqrt(mean_squared_error(Y_test, cnn_lstm_test_prediction)))

Train Error (RMSE): 0.03451280915229757
Test Error (RMSE): 0.10186385943253083


In [ ]:
#save to csv
date_format = "%Y-%m-%d"
a = datetime.strptime(train['Date'].max(), date_format)
a = a + timedelta(7)

next_date = str(datetime.date(a))
next_sales = np.round(cnn_lstm_train_prediction[len(cnn_lstm_train_prediction) - 1], 2)

dataset = dataset.append(pd.DataFrame({'Store' : '1',
                                       'Item' : '1',
                                       'Date' : next_date,
                                       'Weekly_Sales' : next_sales}))
dataset.tail()

In [ ]:
dataset.to_csv('sales data-set.csv', index = False)